In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [16]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [17]:
trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

In [18]:
classes = trainset.classes

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 15 * 15, 128)
        self.fc2 = nn.Linear(128, 2)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 15 * 15)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [23]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
epochs = 10

In [24]:
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100}')
            running_loss = 0.0

In [25]:
testset = torchvision.datasets.ImageFolder(root='./data/test', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True)

In [28]:
total = 0
correct = 0


with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(2)))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')



Predicted:  queue queue
Predicted:  queue queue
Predicted:  noqueue queue
Predicted:  noqueue queue
Predicted:  noqueue queue
Predicted:  noqueue noqueue
Predicted:  queue noqueue
Predicted:  noqueue noqueue
Predicted:  queue noqueue
Predicted:  noqueue queue
Predicted:  queue queue
Predicted:  noqueue queue
Predicted:  noqueue queue
Predicted:  queue queue
Predicted:  queue queue
Predicted:  noqueue noqueue
Predicted:  noqueue noqueue
Predicted:  queue queue
Predicted:  noqueue queue
Predicted:  queue queue
Predicted:  queue queue
Predicted:  queue noqueue
Predicted:  noqueue queue
Predicted:  queue noqueue
Predicted:  queue queue
Predicted:  queue queue
Accuracy: 100.0%


In [29]:
torch.save(net.state_dict(), './model.pth')